# SmartBeds - Preprocesado de One-Class

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [1]:
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
from transformers import *
import pickle as pk

### Carga de los datos

In [2]:
with open('data/crisis_separadas.pkl','rb') as f:
    crisis = pk.load(f)

In [3]:
crisis[0].head()

,DateTime,P1,P2,P3,P4,P5,P6,target
4089271,2018-11-09 21:03:27,4.098361,18.852459,21.311475,22.131148,13.114754,6.557377,False
4089270,2018-11-09 21:03:27,4.098361,18.852459,22.131148,22.131148,13.114754,6.557377,False
4089272,2018-11-09 21:03:28,4.918033,19.672131,22.131148,22.950820,13.934426,7.377049,False
4089273,2018-11-09 21:03:29,4.098361,18.852459,21.311475,22.131148,13.114754,6.557377,False
4089274,2018-11-09 21:03:30,0.000000,18.852459,22.131148,22.950820,13.114754,6.557377,False


### Transformación de los datos

In [4]:
#Según los resultados de alicia la doble ventana de 90 es la más adecuada 
#considerando como crisis solamente los estadísticos que estén formados solo por crisis
nor = Normalizer(max_=100)
wekaAVG = PipelineTransformer(StatisticsTransformer(mode='mean',window=90),nor)
wekaSTD = PipelineTransformer(StatisticsTransformer(mode='std',window=90),nor)
wekaCON = ConcatenateTransformer(wekaAVG,wekaSTD)
wekaTMV = MoveTargetsTransformer(window=90, mode='only')

In [5]:
dias_crisis_pre = []
for d in crisis:
    datetimes = d[['DateTime']]
    targets = d[['target']]
    data = d[['P1','P2','P3','P4','P5','P6']]
    datos = wekaCON.fit_transform(data)
    datos = pd.concat([datetimes,datos,targets],axis=1).dropna()
    dias_crisis_pre.append(wekaTMV.fit_transform(datos))

In [6]:
dias_crisis_pre[2].head()

,DateTime,P1 mean 90,P2 mean 90,P3 mean 90,P4 mean 90,P5 mean 90,P6 mean 90,P1 std 90,P2 std 90,P3 std 90,P4 std 90,P5 std 90,P6 std 90,target
2164173,2019-02-06 21:13:01,0.0,61.502494,77.369077,65.305486,18.578554,0.0,0.0,11.244621,16.810018,18.287520,10.919490,0.0,False
2164174,2019-02-06 21:13:02,0.0,61.627182,77.182045,64.869077,18.391521,0.0,0.0,10.913977,16.392011,15.644671,9.997791,0.0,False
2164175,2019-02-06 21:13:03,0.0,61.689526,76.995012,64.619701,18.329177,0.0,0.0,10.741918,16.127445,14.924277,9.954776,0.0,False
2164176,2019-02-06 21:13:04,0.0,61.596010,76.714464,64.432668,18.360349,0.0,0.0,10.672268,15.009593,14.344971,9.952936,0.0,False
2164177,2019-02-06 21:13:04,0.0,61.596010,76.433915,64.152120,18.329177,0.0,0.0,10.672268,13.459404,12.587417,9.859695,0.0,False


In [7]:
crisis = 1
for d in dias_crisis_pre:
    d.to_csv('data/crisis-'+str(crisis)+'.csv',index=False)
    crisis += 1
total = pd.concat(dias_crisis_pre,axis=0)
total = total.sort_values(by='DateTime')
total.to_csv('data/todas-las-crisis.csv',index=False)

In [8]:
#Dos primeras crisis
dos_pre = pd.concat((dias_crisis_pre[0],dias_crisis_pre[1]),axis=0)
dos_pre.sort_values(by='DateTime')
dos_pre.to_csv('data/dos-primeras-crisis.csv',index=False)